KNN can be used for document retrieval.

For each docs, find bag-of-words or TF-IDF and then use KNN to identify the closest match

**Bag-of-words** : 

1. Counts the number of words in each document and use that to calculate the distance metrics
2. Gives more importance of commnon words and less importance to rare words.
3. Rare words specific to that article but not present in other articles could be more important, as that is the main aspect.
4. Ex : An article on Messi, would be specific in a pool of articles in library. With bag-of-words we may not be able distinguish that

**TF-IDF**:

Term Frequency: Count in that article (Common Locally)              
Inverse document Frequency : Rare Globally
$$ IDF = log \frac{Num docs}{1 + Docs using that word} $$
TF-IDF : Trade-off btw local frequency vs global rarity

# Distance Metrics

## Euclidean Distance      
Shortest distance : | a-b |

But with multiple dimensions, we want weight each dimension differently. Bcos, each feature might have diff scales and diff importance. Ex : Title, abstract are more importance than body of article.

**Scaled Euclidean Distance**
Distance between two features are given body
$$distance(x_i, x_q) = \sqrt {(a_1(x_i[1] - x_q[1])^2 + .. + a_1(x_i[d] - x_q[d])^2)} $$
where 1 represents values of feature1 and $a_1$ is weight on feature1. If set to binary (0, 1), we are including or excluding a feature for distance caluclation, which is part of feature selection.  

$$distance(x_i, x_q) = \sqrt {(X_i - X_q)^TA(X_i - X_q)}$$
This is in vectorized form, where A is weight matrix

## Cosine Similarity:
$$ CS = \frac{X_i^T X_q}{||X_i|| ||X_q||}$$

Where the denominator norm is the magnitude of that vector $ \sqrt{(\sum (X_i)^2)} $. This is used to normalize the vector.

Not a proper distance metric, but efficient to compute spare matrix (bcos u need to calculate dot for only non-zeros)
Called cosine because  of forumaltion ab = |a||b|cos()

If two points are close, then the vector from origin will also be close, indeed the cosine angle will be nearly zero, indicating higher Similarity.

Cosine ingeneral ranges from -1 to 1. But when we keep the tf-idf to positive features then we typically get CS between 0 to 1

Distance = 1 - Cosine Similarity

normalize is to ensure we get same results even if pages are duplicated (ex). We are more focussed on content than number of occurances. CS is invariant to length of document

But CS just can also say a short tweet is very similar to the large document, which can be undesirable. Hence, people typically capp max word counts!

ED is very much proportional to CS if normalized word counts with unit length.

## Example :
In example from wikipedia articles, we are trying to find NN for "Barack Obama"     


Findings:       


1. Bag-of-words & KNN:  

    1.1 This gave results of contemporary presidents as NN for Obama, but not many who worked with Obama.       
    1.2 Reason was the top common words were "the, many, is, etc".      
    1.3 But despite those dominating, the NN werent musicians bcos it gave some level of importance to president related rare words         


2. Importance to Rare Words : TF-IDF     
   
    2.1 TF-IDF penalizes common words, eliminates noise from common words.      
    2.2 Using this, results became better with NN being many who worked with Obama.             
    2.3 NN didnt choose Biden though! Uptil now, distance metric was "euclidean distance".  ED favours short articles over long ones. Or it favours similar length articles. TF-IDF is proportional to article length. Longer the article, higher the TF-IDF weight, bcos more words and vectors.        
    2.4 There is no reason to favor long or short articles in finding NN    
    2.5 To avoid this, Cosine Similarity compares word distributions of varying lengths.            
    2.6 Build TF-IDF, get NN using KNN but with cosine similarity as distance metric. This yield much better NN    
           2.7 It gave Biden, H Clinton, etc.  
    2.8 **Moral of the story**: In deciding the features and distance measures, check if they produce results that make sense for your particular application.          
    2.9 Cosine Similarity ignores all document length totally, which can cause a tweet similarity to a book article!        
    2.10 This can be avoided by enforcing max or min document lengths.      



    


# KNN Problems : 

KNN complexity increases with number of documents or data. You need search through all documents, get their distance to find the nearest neighbour

1-NN - O(N)         
K-NN - O(Nlog(K))

Enter KD-tree

# KD-Tree

The idea is split the entire data into segments  (like tree) and perform KNN on subset of data rather than entire space.

![title](Images\KD_Trees_1.PNG)

During tree construction at every node store:   
1. What feature was split on    
2. feature split value  
3. Smallest Bounding Box that contains all pts

Choices:        
1. Which dimension to split?        
    Widest dimension        
2. Which value to split?    
    Median (or center of box)   
3. When to stop?    
    Min num of points (threshold)


## NN with KD-Tree

From the prediction value, you traverse along the tree. Reach the leaf and calculate the distances in that box.

![title](Images\KD_Trees_Prediction_1.PNG)

Are we done? Not bcos, there are other data points closer than choosen NN. We typically go ahead with finding distances from bounding box and keep pruning,
Ultimately, we tend to calculate less number of distance calculations wrt to data points, bcos we calculate distance to bb.

In below ex, we started with query point, intially choose NN as 0, then kept on pruning those BB which had higher distance than 0, ultimately updating to point 3.

![title](Images\KD_Trees_Prediction_2.PNG)

Problem with KD-trees : at high dimension it becomes cumbersome to prune the trees. Mainly bcos at high dimesion, many dimension can overlap causing inefficieny in pruning.

![title](Images\KD_High_Dimension.PNG)

So approximate neighbour finding: Its okay for finding pretty close neighbour than the cloest neighbour

## Locality Sensitive Hashing (LSH)
For approximate nearest neighbor search

You create a hash table which is like the classification predictions (1, 0). This is created by bins.
Now during prediction, first through binning identify which hash table it belong to (say ex: 1), now perform NN search for data only in "1" category data points

Potential Issues with LSH:  
1. Finding good line to split/bin?  
2. Points close can get split into separate bins    
3. Bin if contain large data, thats still equivalent to NN in computational cost  

3 can be addressed with creating more bins, so lower number of data/bin. We are trading-off accuracy of finding the NN for speed. Bcos we are finding the closer and not closest.       
1 & 2 can be solved by using random line approach! it aint that bad bcos

![title](Images\LSH_Random_Line.PNG)

Searching Neighbour bins:
Few can tend to search with adjacent bins for finding the NN rather oly within the bin
The quality of retrieved NN can only improve with searching more and more neighbouring bins

## LSH Algorithm:
1. Draw h random lines      
2. Compute score for each point binned between lines.     
3. This creates a hash vector for each bin, use this to predict